In [1]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


CONTENT-BASED RECOMENDATION SYSTEM USING BOW AND TF_IDF MODELS

In [11]:
import warnings
import requests
from Utils import *
import pandas as pd
from collections import Counter 
from BagOfWordsModel import BagOfWordsModel
from TFIDFModel import TFIDFModel

In [12]:
def Get_And_Visualize_Data():
    data_set = pd.read_json('2pyij-qh9lc.json')
    print(" No. of Products (or data points) : ",data_set.shape[0],"\n No. of features: ",data_set.shape[1])
    print(data_set.columns) # total features 
    # Features we are gonna use :
    # brand,color,product_type_name,medium image url,title,formatted_price
    data_set = data_set[['nama_villa','jumlah_bedroom','villa_deskripsi','lokasi','image_villa-src','location_deskripsi','bedrooms_deskripsi']]
    # ASIN - Amazon consumer API--as an associate--asin(Amazon Standard Identification No.) - ID / primary key
    return data_set

In [13]:
data_set=Get_And_Visualize_Data()

 No. of Products (or data points) :  827 
 No. of features:  14
Index(['web-scraper-order', 'web-scraper-start-url', 'villa-link',
       'villa-link-href', 'nama_villa', 'jumlah_bedroom', 'villa_deskripsi',
       'bedrooms_deskripsi', 'outdoor_deskripsi', 'indoor_deskripsi',
       'more_deskripsi', 'location_deskripsi', 'lokasi', 'image_villa-src'],
      dtype='object')


In [14]:
def ExploratoryDataAnalysis(data_set):
    data_set.head()
    for feature in data_set.columns:
        print("\n\n\nSTATISTICS FOR FEATURE : ",feature.upper())
        print("\nDescription for Feature : \n")
        print(data_set[feature].describe())
        print("\nCount/Frequency for Feature : \n")
        data_set['nama_villa'].unique()
        print(Counter(list(data_set[feature])).most_common(10))

In [15]:
ExploratoryDataAnalysis(data_set)





STATISTICS FOR FEATURE :  NAMA_VILLA

Description for Feature : 

count                  827
unique                 823
top       Villa Yudhistira
freq                     2
Name: nama_villa, dtype: object

Count/Frequency for Feature : 

[('Villa Cantik', 2), ('Villa Matahari', 2), ('Villa Jasmine', 2), ('Villa Yudhistira', 2), ('Sungai Jungle Villa I', 1), ('Morabito Art Cliff', 1), ('Villa Vinila Penthouse', 1), ('Villa Sally', 1), ('Villa Ibiza', 1), ('Villa Jeruk 1', 1)]



STATISTICS FOR FEATURE :  JUMLAH_BEDROOM

Description for Feature : 

count            827
unique            15
top       3 bedrooms
freq             211
Name: jumlah_bedroom, dtype: object

Count/Frequency for Feature : 

[('3 bedrooms', 211), ('4 bedrooms', 188), ('5 bedrooms', 122), ('2 bedrooms', 104), ('1 bedroom', 73), ('6 bedrooms', 53), ('7 bedrooms', 29), ('8 bedrooms', 16), ('9 bedrooms', 11), ('10 bedrooms', 7)]



STATISTICS FOR FEATURE :  VILLA_DESKRIPSI

Description for Feature : 

count      8

In [23]:
data_set=Data_Preprocessing(data_set)
#data_set=pd.read_pickle('Pickles/Preprocessed_Data_version2')
data_set

def Data_Preprocessing(data_set):
    # descending order of  of Null values in prices>color>brand
    data_set=RemoveNullValues(data_set)
    # Recommendtaion based on different size or color not a good recommendation
    data_set=RemoveDuplicateItemsAfterSorting(data_set)
    print(data_set.shape[0])
    data_set.to_pickle('Pickles/Preprocessed_Data_version1')
    data_set=RemoveDuplicateItemsNonAdjacent(data_set)
    data_set.to_pickle('Pickles/Preprocessed_Data_version2')
    return data_set

 
def RemoveNullValues(data_set):
    hav_null=['nama_villa','jumlah_bedroom']
    for features in hav_null:
        data_set=data_set.loc[-data_set[features].isnull()]
        print("\n\nNo. of data points after eliminating {}=NULL :\n".format(features),data_set.shape[0])
    return data_set

#Near Duplicate items remain
#remove data differing only at end(that is size difference or colour difference are not good recommendations)
def RemoveDuplicateItemsAfterSorting(data_set):
    print("\nThe duplicated items are : ",sum(data_set.duplicated('nama_villa')),"\n\n")
    #sort on the basis of the title(alphabetical order)
    data_set.sort_values('nama_villa',inplace=True,ascending=False)
    #print(data_set['title'].head(20))
    
    row_wise_str_index=[]
    for row_index,row in data_set.iterrows():
        row_wise_str_index.append(row_index)
    new_data_list=[]
    i=0
    j=0
    len_data_points = data_set.shape[0]
    while i<len_data_points and j<len_data_points:
        temp_i=i
        #wordlist for ith string
        i_str = data_set['nama_villa'].loc[row_wise_str_index[i]].split()
        j=i+1
        while j<len_data_points:
            j_str = data_set['nama_villa'].loc[row_wise_str_index[j]].split()
            max_len=max(len(i_str),len(j_str))
            count=0 #no. of words matching in both strings
            #itertools.zip_longest(i_str,j_str) return correspoinding words in format [('a1','b1'),('a2','b2'),(None,'b3')]
            for words in itertools.zip_longest(i_str,j_str):
                if words[0] == words[1]:
                    count+=1
            if(max_len-count>2):
                #word difference>2 both different items -- hence include
                new_data_list.append(data_set['nama_villa'].loc[row_wise_str_index[j]])
                if(max_len-1==j):
                    #word difference>2 but in len_data_points and len_data_points-1
                    new_data_list.append(data_set['nama_villa'].loc[row_wise_str_index[j]])
                i=j
                break
            else:
                j+=1
        if temp_i == i:
            break   
    data_set=data_set.loc[data_set['nama_villa'].isin(new_data_list)]
    return data_set

def RemoveDuplicateItemsNonAdjacent(data_set): # Time complexity = O(n^2) 
    row_wise_str_index=[]
    for row_index,row in data_set.iterrows():
        row_wise_str_index.append(row_index)
    new_data_list=[]
    while(len(row_wise_str_index)!=0):
        #remove the last element in the row_wise_str_index and return the index of the last element
        i = row_wise_str_index.pop()
        new_data_list.append(data_set['nama_villa'].loc[i])
        i_str=data_set['nama_villa'].loc[i].split()
        for j in row_wise_str_index:
            j_str = data_set['nama_villa'].loc[j].split()
            max_len = max(len(i_str),len(j_str))
            count = 0 #count of mathing words in the two strings
            for words in itertools.zip_longest(i_str,j_str):
                if words[0] == words[1]:
                    count+=1
            # word difference > 3 the words are considered different 
            if (max_len-count) < 3:
                row_wise_str_index.remove(j)
    data_set=data_set.loc[data_set['nama_villa'].isin(new_data_list)]
    return data_set




No. of data points after eliminating nama_villa=NULL :
 827


No. of data points after eliminating jumlah_bedroom=NULL :
 827

The duplicated items are :  4 




KeyError: 'web-scraper-order'

In [ ]:
#data_set=TextPreprocessing(data_set)
data_set=pd.read_pickle('Pickles/Preprocessed_Data_version3')
data_set

In [ ]:
BagOfWordsModel(data_set)

In [ ]:
TFIDFModel(data_set)